In [34]:
import torch
import numpy as np
from ogb.nodeproppred import DglNodePropPredDataset
import os
from os.path import exists
import dgl
import time 

from os.path import exists

def get_data_dir():
    import os
    username = os.environ['USER']
    DATA_DIR = None
    if username == 'spolisetty_umass_edu':
        DATA_DIR = "/work/spolisetty_umass_edu/data"
    if username == "spolisetty":
        DATA_DIR = "/data/sandeep"
    if username == "q91":
        DATA_DIR = "/mnt/bigdata/sandeep"
    if username == "ubuntu":
        # AWS 
        DATA_DIR = "/home/ubuntu/data"     
    assert(DATA_DIR != None)
    return DATA_DIR

ROOT_DIR = get_data_dir()

In [35]:
name = "ogbn-arxiv"

TARGET_DIR = ROOT_DIR +"/" + name

dataset = DglNodePropPredDataset(name, root=ROOT_DIR)
graph, labels = dataset[0]
edges = graph.edges()
graph.remove_edges(torch.where(edges[0] == edges[1])[0])
edges = graph.edges()
num_edges = graph.num_edges()
num_nodes = graph.num_nodes()
assert(num_edges == edges[0].shape[0])
features = graph.ndata['feat']
assert features.shape[0] == num_nodes

split_idx = dataset.get_idx_split()
train_idx = split_idx['train']
val_idx = split_idx['valid']
test_idx = split_idx['test']

print("Train", train_idx.shape)
print("Val", val_idx.shape)
print("Test", test_idx.shape)


Train torch.Size([90941])
Val torch.Size([29799])
Test torch.Size([48603])


In [36]:
num_partitions = 4
p_map = (np.fromfile(TARGET_DIR + f'/partition_map_opt_{num_partitions}.bin', dtype = np.int32))
partition_map = torch.from_numpy(p_map)


In [46]:
 new_order = None
degrees = graph.out_degrees()
ordered_partition_nodes = []
partition_offsets = [0]
for i in range(num_partitions):
    nodes_in_partition = torch.where(partition_map == i)[0]
    _, indices = torch.sort(degrees[nodes_in_partition], descending = True)
    ordered_partition_nodes.append(nodes_in_partition[indices])
    partition_offsets.append(partition_offsets[-1] + indices.shape[0])
    
new_to_old_order = torch.cat(ordered_partition_nodes, dim = 0)    
values, old_to_new_order = torch.sort(new_to_old_order)
assert(torch.all(torch.arange(graph.num_nodes()) == values))

In [48]:
labels = labels.flatten()
print(labels[train_idx][:10])
new_labels = labels[new_to_old_order]
new_train_idx = old_to_new_order[train_idx]
print(new_labels[new_train_idx][:10])

tensor([ 4,  5, 28,  8, 27, 34,  6,  4,  3, 28])
tensor([ 4,  5, 28,  8, 27, 34,  6,  4,  3, 28])
